# FINAL PROJECT - OCCASION VEHICLES PRICE ESTIMATOR
___
#### MASTER IN DATA SCIENCE - KSCHOOL - 2016/17
#### KOLDO PINA ORTIZ
____

## Motivation

The aim of the present work is to get a good estimator of the price of a second-hand vehicle, based on the prices of the second-hand market

## Action Plan

To reach our goal, we will follow the the next steps:

1. ***Scrape*** both websites, **motos.net** and **coches.net**, to obtain the ***data***.
2. ***Merge*** and ***clean*** the ***data***.
3. ***Train*** various models.
4. Compare the metrics and choose the model with the best one.
5. Create a flask web server with the app.

## Scrape

We are going to scrape each website separately.
To this end, we have developed two python scripts called ***scraper_motos.py*** and ***scrapers_coches.py***.
Both return a dataframe.

You can import them to the notebook.

In [ ]:
#Call the function to scrape motos.net and create a csv file with the raw data
#from scraper_motos import scraper_motos
#scraper_motos().to_csv('motos_raw_data.csv', index = False, header = True, encoding = 'utf-8')

In [ ]:
#import pandas as pd
#df_motos = pd.read_csv('motos_raw_data.csv')